In [2]:
import math

In [3]:
import pandas as pd
import numpy as np
import keras
import os
import re
import math
# from google.colab import drive


from keras.layers import Hashing
from keras.utils import pad_sequences

from sklearn.model_selection import train_test_split

from sklearn.metrics import confusion_matrix,classification_report,accuracy_score,recall_score,precision_score,f1_score


: 

: 

In [2]:
# # changes directory from colab to gdrive
# drive.mount('/content/drive', force_remount=True)

# base_path = '/content/drive/MyDrive/INFO251_Fake_Job_Detection/'
# os.chdir(base_path)
# baseline_csv = './Data/text.csv'  # data dictionary
# data = os.path.join(base_path, baseline_csv)
# dfRaw = pd.read_csv(data)

In [2]:
dfRaw = pd.read_csv('./../Data/text.csv')

NameError: name 'pd' is not defined

# Data processing and construct traning/set set

In [4]:
def textProcessing(string):
  ret = re.sub('[^a-zA-Z]',' ',string).split()
  ret = ' '.join(ret)
  return ret 
textRaw = dfRaw.text.apply(lambda x: textProcessing(x)).to_list()

## Check the total size of unique words to determine how many bits are needed in onehot encoding

In [5]:
unique = set()
for t in textRaw:
  unique.update(set(t.split()))
unique = list(unique)
n_words = math.ceil(len(unique))
onehot_hasher = Hashing(num_bins=n_words,output_mode='one_hot')
onehot_map = onehot_hasher(unique)
onehot_dict = dict(zip(unique,onehot_map))
# turn pure onthot into onehot index for keras embeding layer
textOnehot = np.array([np.array([int(onehot_dict[x].numpy().argmax()) for x in t.split()]) for t in textRaw])


C:\Users\Eric\AppData\Local\Temp\ipykernel_3768\404515473.py:10: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  textOnehot = np.array([np.array([int(onehot_dict[x].numpy().argmax()) for x in t.split()]) for t in textRaw])


## Train-test split

In [6]:
max_sequence_length = 500
X = pad_sequences(textOnehot, maxlen=max_sequence_length)
# also encode y
# t - fraud - 1
# f - nonfraud - 0
y = dfRaw.fraudulent.apply(lambda x: 1 if x=='t' else 0).to_numpy()
assert(len(X)==len(y))
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.25,random_state=42)

# Bidirectional LSTM

In [8]:
from keras.layers import Embedding,LSTM,Bidirectional,Dropout,Dense
from keras.models import Sequential
import random
import numpy as np
import tensorflow as tf
seed=42
random.seed(seed)
np.random.seed(seed)
tf.random.set_seed(seed)

## Hyper-parameter tunning LSTM

In [30]:
 
def LSTM_Runner(X_train,y_train,X_test,y_test,embeding_size=50,dropout_r=0.1,LSTM_layer=1,class_weight=[1.0,1.0]):
    embedding_vector_features_size=embeding_size
    LSTMmodel=Sequential()
    LSTMmodel.add(Embedding(n_words,embedding_vector_features_size,input_length=max_sequence_length))
    for _ in range(LSTM_layer-1):
        LSTMmodel.add(Bidirectional(LSTM(100, return_sequences=True)))
    LSTMmodel.add(Bidirectional(LSTM(100)))
    LSTMmodel.add(Dropout(dropout_r))
    LSTMmodel.add(Dense(1,activation='sigmoid'))
    LSTMmodel.compile(loss='binary_crossentropy',optimizer='adam',metrics=['AUC','Recall','Precision'])
    print(f'model initialized')
    model = LSTMmodel
            
    print(f'model fitting with {embeding_size}，{LSTM_layer}, {class_weight}')
    model.fit(
        X_train,
        y_train,
        validation_split=0.2,
        epochs=5,
        batch_size=64,
        class_weight = {0:class_weight[0],1:class_weight[1]},
        verbose=1
    )
    print(f'model fitted')

    y_pred = model.predict(X_test)
    y_pred[y_pred>=0.5]=1
    y_pred[y_pred<0.5]=0

    tn, fp, fn, tp =confusion_matrix(y_pred,y_test).ravel()
    cmatrix = (pd.DataFrame.from_records([[tp,fp],[fn,tn]],columns=['Positive','Neagtive'],index=['Positive','Negative']))
    accu = accuracy_score(y_pred,y_test)
    recall = recall_score(y_pred,y_test)
    prec = precision_score(y_pred,y_test)
    return ([accu,recall,prec])

In [28]:
dfModelCompare = pd.DataFrame(columns=['name','embeding_size','LSMT_layer','weight_0','weight_1','accu','recall','prec'])

embeding_size = [50,100,200]
# dropout_r = [0.1,0.2,0.3,0.5,0.6]
LSTM_layer = [1,2,3]
class_weight = [[1.0,1.0],[1.0,2.0],[1.0,3.0]]
# em_size = 100
# r = 0.3
# l = 1
# w = class_weight[0]
# LSTM_Runner(X_train,y_train,X_test,y_test)
# results = LSTM_Runner(X_train,y_train,X_test,y_test,em_size,0.1,l,w)

In [31]:
cnt = 0
for em_size in embeding_size:
    for l in LSTM_layer:
        for w in class_weight:
            if cnt < 4:
                cnt+=1
                continue
            results = LSTM_Runner(X_train,y_train,X_test,y_test,em_size,0.1,l,w)
            cnt+=1
            dfModelCompare.loc[len(dfModelCompare.index)] = [cnt,em_size,l]+w+results
            print(cnt,' | ',em_size,' | ',l,' | ',w)
            dfModelCompare.to_csv('./Model_Compare2.csv')


model initialized
model fitting with 50，2, [1.0, 2.0]
Epoch 1/5
168/168 [==============================] - 342s 2s/step - loss: 0.2498 - auc: 0.8208 - recall: 0.2798 - precision: 0.5779 - val_loss: 0.1071 - val_auc: 0.9288 - val_recall: 0.5899 - val_precision: 0.7009
Epoch 2/5
168/168 [==============================] - 322s 2s/step - loss: 0.0847 - auc: 0.9786 - recall: 0.8016 - precision: 0.8145 - val_loss: 0.0961 - val_auc: 0.9354 - val_recall: 0.5827 - val_precision: 0.8265
Epoch 3/5
168/168 [==============================] - 511s 3s/step - loss: 0.0350 - auc: 0.9915 - recall: 0.9345 - precision: 0.9146 - val_loss: 0.1135 - val_auc: 0.9317 - val_recall: 0.6978 - val_precision: 0.6178
Epoch 4/5
168/168 [==============================] - 577s 3s/step - loss: 0.0195 - auc: 0.9952 - recall: 0.9643 - precision: 0.9624 - val_loss: 0.1204 - val_auc: 0.9081 - val_recall: 0.6547 - val_precision: 0.6947
Epoch 5/5
168/168 [==============================] - 512s 3s/step - loss: 0.0119 - auc: 0.

## Find the best Parameter and save the model.

In [35]:
X_train.shape

(13410, 500)

In [38]:
BEST_EMBEDING = 50
BEST_LSTM_LAYER = 3
BEST_WEIGHT = [1.0,2.0]
MAX_SEQUENCE_LENGTH = 500


embedding_vector_features_size=BEST_EMBEDING
LSTMmodel=Sequential()
LSTMmodel.add(Embedding(n_words,embedding_vector_features_size,input_length=MAX_SEQUENCE_LENGTH))
for _ in range(BEST_LSTM_LAYER-1):
    LSTMmodel.add(Bidirectional(LSTM(100, return_sequences=True)))
LSTMmodel.add(Bidirectional(LSTM(100)))
LSTMmodel.add(Dropout(0.1))
LSTMmodel.add(Dense(1,activation='sigmoid'))
LSTMmodel.compile(loss='binary_crossentropy',optimizer='adam',metrics=['AUC','Recall','Precision'])
print(f'model initialized')
model = LSTMmodel
        
print(f'model fitting with weight {BEST_WEIGHT}')
model.fit(
    X_train,
    y_train,
    validation_split=0.2,
    epochs=5,
    batch_size=64,
    class_weight = {0:BEST_WEIGHT[0],1:BEST_WEIGHT[1]},
    verbose=1
)
print(f'model fitted')


model initialized
model fitting with weight [1.0, 2.0]
Epoch 1/5
168/168 [==============================] - 537s 3s/step - loss: 0.2329 - auc: 0.8314 - recall: 0.2996 - precision: 0.5261 - val_loss: 0.1028 - val_auc: 0.9227 - val_recall: 0.4964 - val_precision: 0.9079
Epoch 2/5
168/168 [==============================] - 641s 4s/step - loss: 0.0795 - auc: 0.9788 - recall: 0.7917 - precision: 0.8507 - val_loss: 0.1067 - val_auc: 0.9391 - val_recall: 0.5971 - val_precision: 0.8137
Epoch 3/5
168/168 [==============================] - 552s 3s/step - loss: 0.0285 - auc: 0.9910 - recall: 0.9425 - precision: 0.9443 - val_loss: 0.1222 - val_auc: 0.9286 - val_recall: 0.7050 - val_precision: 0.6447
Epoch 4/5
168/168 [==============================] - 501s 3s/step - loss: 0.0126 - auc: 0.9976 - recall: 0.9802 - precision: 0.9821 - val_loss: 0.1541 - val_auc: 0.8674 - val_recall: 0.6331 - val_precision: 0.7521
Epoch 5/5
168/168 [==============================] - 474s 3s/step - loss: 0.0042 - auc: 1

INFO:tensorflow:Assets written to: ./Model/LSTM_MODEL/assets


INFO:tensorflow:Assets written to: ./Model/LSTM_MODEL/assets


In [44]:
import tensorflow as tf
tf.keras.utils.plot_model(model)


You must install pydot (`pip install pydot`) and install graphviz (see instructions at https://graphviz.gitlab.io/download/) for plot_model to work.


In [39]:

saved_model_path = './Model/LSTM_MODEL/pre-trained-LSTM'.format('')
model.save(saved_model_path, include_optimizer=False)

You must install pydot (`pip install pydot`) and install graphviz (see instructions at https://graphviz.gitlab.io/download/) for plot_model to work.


INFO:tensorflow:Assets written to: ./Model/LSTM_MODEL/assets


INFO:tensorflow:Assets written to: ./Model/LSTM_MODEL/assets


## Evaluation

In [46]:
BEST_LSTM_fp = './Model/LSTM_MODEL/'
BEST_MODEL = tf.keras.models.load_model(BEST_LSTM_fp)

In [51]:
BEST_MODEL.compile(loss='binary_crossentropy',optimizer='adam',metrics=['AUC','Recall','Precision'])
BEST_MODEL.evaluate(X_test,y_test)

140/140 [==============================] - 106s 691ms/step - loss: 0.1475 - auc: 0.8859 - recall: 0.6951 - precision: 0.8073


[0.1474790871143341, 0.8858898878097534, 0.695067286491394, 0.8072916865348816]

In [63]:
y_pred = BEST_MODEL.predict(X_test)

140/140 [==============================] - 93s 663ms/step


In [64]:
y_pred = np.round(y_pred).astype(int)

In [65]:
accu = accuracy_score(y_pred,y_test)
recall = recall_score(y_pred,y_test)
prec = precision_score(y_pred,y_test)

print(f'Accuracy: {accu:.4f}')
print(f'Recall: {recall:.4f}')
print(f'Precision: {prec:.4f}')

Accuracy: 0.9765
Recall: 0.8073
Precision: 0.6951
